# Polynomial Optimization

**Contributed by**: Benoît Legat

## Introduction

Consider the polynomial optimization problem [L09, Example 2.2] of
minimizing the polynomial $x^3 - x^2 + 2xy - y^2 + y^3$
over the polyhedron defined by the inequalities $x \ge 0, y \ge 0$ and $x + y \geq 1$.

[L09] Lasserre, J. B.
*Moments, positive polynomials and their applications*.
World Scientific, **2009**.

In [1]:
using DynamicPolynomials
@polyvar x y
p = x^3 - x^2 + 2x*y -y^2 + y^3
using SumOfSquares
S = @set x >= 0 && y >= 0 && x + y >= 1
p(x=>1, y=>0), p(x=>1//2, y=>1//2), p(x=>0, y=>1)

(0, 1//4, 0)

## Local search

A local solver only uses the **local** information given by the the value, gradient and hessian
of the objective function and constraints at a given solution. When it converges, it therefore only
guarantees that the found solution is a **local** minimum.
In this example, the optimal solutions are $(x, y) = (1, 0)$ and $(x, y) = (0, 1)$ with objective value $0$ but
[Ipopt](https://github.com/jump-dev/Ipopt.jl/) only finds the local minimum $(1/2, 1/2)$ with objective value $1/4$.

In [2]:
import Ipopt
model = Model(Ipopt.Optimizer)
@variable(model, a >= 0)
@variable(model, b >= 0)
@constraint(model, a + b >= 1)
@NLobjective(model, Min, a^3 - a^2 + 2a*b - b^2 + b^3)
optimize!(model)

Iter: 12 Ap: 9.00e-01 Pobj: -6.0000000e+00 Ad: 1.00e+00 Dobj: -6.0000000e+00 
Success: SDP solved
Primal objective value: -6.0000000e+00 
Dual objective value: -6.0000000e+00 
Relative primal infeasibility: 4.61e-11 
Relative dual infeasibility: 3.44e-11 
Real Relative Gap: 5.36e-10 
XZ Relative Gap: 7.07e-10 
DIMACS error measures: 5.03e-11 0.00e+00 5.84e-11 0.00e+00 5.36e-10 7.07e-10

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in La

As we can see below, the termination status is `LOCALLY_SOLVED` and not of `OPTIMAL`
because Ipopt only guarantees **local** optimality.

In [3]:
solution_summary(model)

* Solver : Ipopt

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "Solve_Succeeded"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 2.50000e-01
  Dual objective value : 7.50000e-01

* Work counters
  Solve time (sec)   : 1.74144e+00


Indeed, the solution found is not globally optimal:

In [4]:
value(a), value(b)

(0.49999999667061656, 0.4999999966705879)

Note that the problem can be written equivalently as follows using [registered functions](https://jump.dev/JuMP.jl/stable/manual/nlp/#Register-a-function).
The difference is that the gradient and hessian will be computed via the *Symbolic Differentiation* provided
by MultivariatePolynomials instead of JuMP's *Automatic Differentiation*:

In [5]:
f(a, b) = p(x => a, y => b)
∇p = differentiate(p, [x, y])
function ∇f(g, a, b)
    for i in eachindex(g)
        g[i] = ∇p[i](x => a, y => b)
    end
end
∇²p = differentiate(∇p, [x, y])
function ∇²f(H, a, b)
    for j in axes(∇²p, 2)
        for i in j:size(∇²p, 1)
            H[i, j] = ∇²p[i, j](x => a, y => b)
        end
    end
end
using Ipopt
gmodel = Model(Ipopt.Optimizer)
@variable(gmodel, a >= 0)
@variable(gmodel, b >= 0)
@constraint(gmodel, a + b >= 1)
register(gmodel, :f, 2, f, ∇f, ∇²f)
@NLobjective(gmodel, Min, f(a, b))
optimize!(gmodel)

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.9999940e-06 9.80e-01 1.33e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

Even if we have the algebraic expressions of gradient and hessian,
Ipopt is not using these symbolic expressions but only local information
hence it can still only provide local guarantees:

In [6]:
solution_summary(gmodel)

* Solver : Ipopt

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "Solve_Succeeded"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 2.50000e-01
  Dual objective value : 7.50000e-01

* Work counters
  Solve time (sec)   : 1.06059e-01


and the same solution is found:

In [7]:
value(a), value(b)

(0.49999999667061656, 0.4999999966705879)

## Sum-of-Squares approach

We will now see how to find the optimal solution using Sum of Squares Programming.
We first need to pick an SDP solver, see [here](https://jump.dev/JuMP.jl/v1.12/installation/#Supported-solvers) for a list of the available choices.

In [8]:
import SCS
scs = SCS.Optimizer
import Dualization
dual_scs = Dualization.dual_optimizer(scs)

#11 (generic function with 1 method)

A Sum-of-Squares certificate that $p \ge \alpha$ over the domain `S`, ensures that $\alpha$ is a lower bound to the polynomial optimization problem.
The following program searches for the largest lower bound and finds zero.

In [9]:
model = SOSModel(dual_scs)
@variable(model, α)
@objective(model, Max, α)
@constraint(model, c3, p >= α, domain = S)
optimize!(model)

------------------------------------------------------------------
	       SCS v3.2.3 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 10, constraints m: 25
cones: 	  z: primal zero / dual free vars: 1
	  s: psd vars: 24, ssize: 4
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 37, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.90e+01  1.01e+00  5.26e+01 -2.61e+01  1.00e-01  2.56e-03 
   125| 4.27e-05  3.97e-05  9.70e-06  8.12e-06  8.63e-01  3.51e-03 
-------------------------------

This time, the termination status is `OPTIMAL` but this does not necessarily mean that we found
the optimal solution to the polynomial optimization problem.
This only means that CSDP founds an optimal solution to the Sum-of-Squares relaxation.

In [10]:
solution_summary(model)

* Solver : Dual model with SCS attached

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.29679e-05
  Dual objective value : 3.27176e-06

* Work counters
  Solve time (sec)   : 3.50879e-03


The feasibility of the primal solution guarantees that the objective value `0` is a lower bound
to the polynomial optimization problem.
The optimality means that it's the best lower bound we can get (at this degree of the hierarcy).
Using the solution $(1/2, 1/2)$ found by Ipopt of objective value $1/4$
and this certificate of lower bound $0$ we know that the optimal objective value is in the interval $[0, 1/4]$
but we still do not know what it is (if we consider that we did not try the solutions $(1, 0)$ and $(0, 1)$ as done in the introduction).
If the dual of the constraint `c3` was atomic, its atoms would have given optimal solutions of objective value $0$ but that is not the case.

In [11]:
ν3 = moment_matrix(c3)
atomic_measure(ν3, 1e-3) # Returns nothing as the dual is not atomic

Fortunately, there is a hierarchy of programs with increasingly better bounds that can be solved until we get one with atom dual variables.
This comes from the way the Sum-of-Squares constraint with domain `S` is formulated.
The polynomial $p - \alpha$ is guaranteed to be nonnegative over the domain `S` if there exists Sum-of-Squares polynomials $s_0$, $s_1$, $s_2$, $s_3$ such that
$$
p - \alpha = s_0 + s_1 x + s_2 y + s_3 (x + y - 1).
$$
Indeed, in the domain `S`, $x$, $y$ and $x + y - 1$ are nonnegative so the right-hand side is a sum of squares hence is nonnegative.
Once the degrees of $s_1$, $s_2$ and $s_3$ have been decided, the degree needed for $s_0$ will be determined but we have a freedom in choosing the degrees of $s_1$, $s_2$ and $s_3$.
By default, they are chosen so that the degrees of $s_1 x$, $s_2 y$ and $s_3 (x + y - 1)$ match those of $p - \alpha$ but this can be overwritten using the `maxdegree` keyword argument.

### The maxdegree keyword argument

The maximum total degree (i.e. maximum sum of the exponents of $x$ and $y$) of the monomials of $p$ is 3 so the constraint in the program above is equivalent to `@constraint(model, p >= α, domain = S, maxdegree = 3)`.
That is, since $x$, $y$ and $x + y - 1$ have total degree 1, the sum of squares polynomials $s_1$, $s_2$ and $s_3$ have been chosen with maximum total degree $2$.
Since these polynomials are sums of squares, their degree must be even so the next maximum total degree to try is 4.
For this reason, the keywords `maxdegree = 4` and `maxdegree = 5` have the same effect in this example.
In general, if the polynomials in the domain are not all odd or all even, each value of `maxdegree` has a different effect in the choice of the maximum total degree of some $s_i$.

In [12]:
model = SOSModel(dual_scs)
@variable(model, α)
@objective(model, Max, α)
@constraint(model, c4, p >= α, domain = S, maxdegree = 4)
optimize!(model)

------------------------------------------------------------------
	       SCS v3.2.3 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 10, constraints m: 25
cones: 	  z: primal zero / dual free vars: 1
	  s: psd vars: 24, ssize: 4
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 37, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.90e+01  1.01e+00  5.26e+01 -2.61e+01  1.00e-01  1.41e-04 
   125| 4.27e-05  3.97e-05  9.70e-06  8.12e-06  8.63e-01  1.04e-03 
-------------------------------

We can see that the basis of the moment matrix didn't increase:

In [13]:
moment_matrix(c4)

MomentMatrix with row/column basis:
 MonomialBasis([1, y, x])
And entries in a 3×3 SymMatrix{Float64}:
 0.9999988622359877  0.4999978058971255    0.4999978058971196
 0.4999978058971255  0.4999205453112076    7.563537032903693e-5
 0.4999978058971196  7.563537032903693e-5  0.499920545311202

This is because of the Newton polytope reduction that determined that gram matrix will
be zero for these degrees so it reduced the problem back to the equivalent of `maxdegree` 3
Let's turn this off with `newton_polytope = nothing`

In [14]:
function sos(solver, deg)
    model = SOSModel(solver)
    @variable(model, α)
    @objective(model, Max, α)
    @constraint(model, c, p >= α, domain = S, maxdegree = deg, newton_polytope = nothing)
    optimize!(model)
    return model
end
dual_model4 = sos(dual_scs, 4)
nothing #hide

------------------------------------------------------------------
	       SCS v3.2.3 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 15, constraints m: 40
cones: 	  z: primal zero / dual free vars: 1
	  s: psd vars: 39, ssize: 4
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 52, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.06e+01  4.30e-01  3.19e+01 -1.57e+01  1.00e-01  3.00e-04 
   125| 2.44e-04  2.14e-05  1.20e-05 -4.90e-05  1.00e-01  2.00e-03 
-------------------------------

We see that the lower bound is still 0:

In [15]:
solution_summary(dual_model4)

* Solver : Dual model with SCS attached

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : -4.29823e-05
  Dual objective value : -5.49868e-05

* Work counters
  Solve time (sec)   : 2.00419e-03


Let's now look at which solution we can extract from the moment matrix:

In [16]:
dual_ν4 = moment_matrix(dual_model4[:c])

MomentMatrix with row/column basis:
 MonomialBasis([1, y, x, y^2, x*y, x^2])
And entries in a 6×6 SymMatrix{Float64}:
  1.0000084649794814       0.5001118271563914      …   0.500333618195221
  0.5001118271563914       0.5003460975047997         -8.01456584878068e-5
  0.5001118271564011      -0.00012664913633449412      0.5004463755228483
  0.5003336181952089       0.5004463755228384          1.95291413604411
 -0.00012729223308665825  -9.194362986997219e-5       -1.8561449853462064
  0.500333618195221       -8.01456584878068e-5     …   8.524707147814102

Looking at the singular values, `4` seems to be a reasonable rank:

In [17]:
using LinearAlgebra
svdvals(Matrix(dual_ν4.Q))

6-element Vector{Float64}:
 11.292137082083745
  6.612780281630302
  1.5788661771503687
  1.0597600660607904
  0.45948547678081636
  2.0487912969849216e-16

The solution we extract is `(0.5, 0.5)` which is the solution found by Ipopt:

In [18]:
atomic_measure(dual_ν4, FixedRank(4))

Atomic measure on the variables x, y with 1 atoms:
 at [0.5000778765242979, 0.5001018243812632] with weight 1.6079422332660906

This process is quite sensitive numerically so let's try to solve it without dualization as well:

In [19]:
model4 = sos(scs, 4)
nothing #hide

------------------------------------------------------------------
	       SCS v3.2.3 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 40, constraints m: 54
cones: 	  z: primal zero / dual free vars: 15
	  s: psd vars: 39, ssize: 4
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 91, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 2.21e+01  1.35e+00  3.74e+01 -1.86e+01  1.00e-01  3.73e-04 
   175| 7.26e-05  8.28e-06  2.10e-05 -1.57e-05  1.00e-01  2.74e-03 
------------------------------

We see that the lower bound is again 0:

In [20]:
solution_summary(model4)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 2.61826e-05
  Dual objective value : 5.20758e-06

* Work counters
  Solve time (sec)   : 2.74199e-03


The moment matrix is the following

In [21]:
ν4 = moment_matrix(model4[:c])

MomentMatrix with row/column basis:
 MonomialBasis([1, y, x, y^2, x*y, x^2])
And entries in a 6×6 SymMatrix{Float64}:
 1.0000007181476611     0.4999993341166773     …  0.4999833790296174
 0.4999993341166773     0.4999834837557172        8.476859131833671e-6
 0.49999933411667985    1.4825415890134371e-5     0.4999724166510302
 0.49998337902961515    0.49997241665102765       0.26228006757456934
 1.5473299845360444e-5  7.3792329870374735e-6     0.2544853069696296
 0.4999833790296174     8.476859131833671e-6   …  0.7968257005643516

Looking at the singular values, `3` seems to be a reasonable rank:

In [22]:
svdvals(Matrix(ν4.Q))

6-element Vector{Float64}:
 2.1989525826314282
 1.0175199095061902
 0.6030514893216119
 0.019380459835785388
 0.01699438182342016
 1.0272412631089054e-16

This time, the dual variable is atomic as it is the moments of the measure
$$0.5 \delta(x-1, y) + 0.5 \delta(x, y-1)$$
where $\delta(x, y)$ is the dirac measure centered at $(0, 0)$.
Therefore the program provides both a certificate that $0$ is a lower bound and a certificate that it is also an upper bound since it is attained at the global minimizers $(1, 0)$ and $(0, 1)$.

In [23]:
atomic_measure(ν4, FixedRank(3))

Atomic measure on the variables x, y with 2 atoms:
 at [0.9946508364409754, 0.00534612538283624] with weight 0.5511753483414487
 at [-0.011834484693154339, 1.0118314465169658] with weight 0.5285747070036433

## A deeper look into atom extraction

The moment matrix is transformed into a system of polynomials equations whose solutions give the atoms.
This transformation uses the SVD decomposition of the moment matrix and discards the equations corresponding to the lowest singular values.
When this system of equation has an infinite number of solutions, `atomic_measure` concludes that the measure is not atomic.
For instance, with `maxdegree = 3`, we obtain the system
$$x + y = 1$$
which contains a whole line of solution.
This explains `atomic_measure` returned `nothing`.

In [24]:
ν3 = moment_matrix(c3)
SumOfSquares.MultivariateMoments.compute_support!(ν3, LeadingRelativeRankTol(1e-3))

With `maxdegree = 4`, we obtain the system
$$
\begin{aligned}
  x + y & = 1\\
  y^2 & = y\\
  xy & = 0\\
  -y + y^2 - x*y & = 0
  y^2 - 2y + 1 & = x^2
\end{aligned}
$$

In [25]:
ν4 = moment_matrix(model4[:c])
SumOfSquares.MultivariateMoments.compute_support!(ν4, FixedRank(3))

This system can be reduced to the equivalent system
$$
\begin{aligned}
  x + y & = 1\\
  y^2 & = y
\end{aligned}
$$
which has the solutions $(0, 1)$ and $(1, 0)$.

In [26]:
SemialgebraicSets.compute_gröbner_basis!(ideal(ν4.support))
ν4.support

Algebraic Set defined by 2 equalities
 -0.9999969618238118 + 1.0000000000000002*y + x = 0
 0.0054093777793762995 - 1.0171775718998022*y + y^2 = 0


The solutions of this system then give the minimizers

In [27]:
collect(ν4.support)

2-element Vector{Vector{Float64}}:
 [0.9946508364409757, 0.005346125382835845]
 [-0.01183448469315476, 1.0118314465169662]

The function `atomic_measure` then reuses the matrix of moments to find the weights $1/2$, $1/2$ corresponding to the diracs centered respectively at $(0, 1)$ and $(1, 0)$.
This details how the function obtained the result
$$0.5 \delta(x-1, y) + 0.5 \delta(x, y-1)$$
given in the previous section.

## HomotopyContinuation

As discussed in the previous section, the atom extraction relies on the solution
of a system of algebraic equations. The `atomic_measure` function takes an optional
`algebraic_solver` argument that is used to solve this system of equation.
If no solver is provided, the default solver of SemialgebraicSets.jl is used which
currently computes the Gröbner basis, then the multiplication matrices and
then the Schur decomposition of a random combination of these matrices.
As the system of equations is obtained from a numerical solution and is represented
using floating point coefficients, homotopy continuation is recommended as it is
more numerically robust than Gröbner basis computation.
The following uses homotopy continuation to solve the system of equations.

In [28]:
using HomotopyContinuation
algebraic_solver = SemialgebraicSetsHCSolver(; excess_residual_tol = 1e-1, real_tol = 1e-1, compile = false)
atomic_measure(ν4, FixedRank(3), Echelon(), algebraic_solver)

Computing mixed cells... 2    Time: 0:00:00
  mixed_volume:  4
Computing mixed cells... 2    Time: 0:00:00
  mixed_volume:  4
Tracking 4 paths...  50%|███████████████▌               |  ETA: 0:00:19
  # paths tracked:                  2
  # non-singular solutions (real):  0 (0)
  # singular endpoints (real):      0 (0)




Tracking 4 paths... 100%|███████████████████████████████| Time: 0:00:19
  # paths tracked:                  4
  # non-singular solutions (real):  0 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         0 (0)


Atomic measure on the variables x, y with 2 atoms:
 at [1.013703462671378, 0.006199097611251542] with weight 0.5304017320803744
 at [0.007317793281148993, 1.0130361715599232] with weight 0.5313320768346611

As the system has 3 equations for 2 variables and the coefficients of the equations
are to be treated with tolerance since they originate from the solution of an SDP,
we need to set `excess_residual_tol` and `real_tol` to a high tolerance otherwise,
HomotopyContinuation would consider that there is no solution.
Indeed, as the system is overdetermined (it has more equations than variables)
HomotopyContinuation expects to have excess solution hence it filters out
excess solution among the solution found. It determines which solution are in excess
by comparing the infinity norm of the residuals of the equations at the solution with `excess_residual_tol`.
It also filters out solution for which the absolute value of the imaginary part of one of the entry
is larger than `real_tol` and strips out the imaginary part.
The raw solutions obtained by HomotopyContinuation can be obtained as follows:

In [29]:
F = HomotopyContinuation.System(ν4.support)
res = HomotopyContinuation.solve(F, algebraic_solver.options...)
path_results(res)

Tracking 4 paths...  50%|███████████████▌               |  ETA: 0:00:03
  # paths tracked:                  2
  # non-singular solutions (real):  0 (0)
  # singular endpoints (real):      0 (0)




Tracking 4 paths... 100%|███████████████████████████████| Time: 0:00:03
  # paths tracked:                  4
  # non-singular solutions (real):  0 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         0 (0)


4-element Vector{HomotopyContinuation.PathResult}:
 PathResult:
 • return_code → :excess_solution
 • solution → ComplexF64[0.3520350941188712 + 0.1438727459720398im, 0.39808373752365767 - 0.29772230033301617im]
 • accuracy → 0.51226
 • residual → 0.52826
 • condition_jacobian → 3.4911
 • steps → 34 / 0
 • extended_precision → false
 • path_number → 1

 PathResult:
 • return_code → :excess_solution
 • solution → ComplexF64[-0.007745883866339952 + 0.019647542972814454im, 1.0375146986833956 - 0.03419024905592798im]
 • accuracy → 0.057842
 • residual → 0.043178
 • condition_jacobian → 7.837
 • steps → 35 / 0
 • extended_precision → false
 • path_number → 2

 PathResult:
 • return_code → :excess_solution
 • solution → ComplexF64[1.04844485683341 + 0.0035136552972948576im, -0.01982184385126279 + 0.009294106102657797im]
 • accuracy → 0.054746
 • residual → 0.038775
 • condition_jacobian → 22.498
 • steps → 49 / 0
 • extended_precision → false
 • path_number → 3

 PathResult:
 • return_code → 

The printed `residual` above shows why `1e-1` allows to filter how the 2 actual
solutions from the 2 excess solutions.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*